In [248]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import codecs
import time
import re
import os
from IPython.display import display, HTML
from difflib import SequenceMatcher

In [249]:
import importlib.util
spec = importlib.util.spec_from_file_location("module.name","sanskrit_transcoder/transcoder.py")
transcoder = importlib.util.module_from_spec(spec)
spec.loader.exec_module(transcoder)
transcoder.transcoder_set_dir('sanskrit_transcoder/data/transcoder');

In [184]:
## here we give path ,name . 
## name we give folder name  we want in path dir

path = "workdata/lines_submit/wrong_lines/sorteddata/"   #  or "workdata/lines_submit/correct_lines/sorteddata/" 
name = "1008"


In [151]:
fname1 = path+name+"/dataframe.txt"                     ## before moodified so we take fname5 instead of it
fname2 = path+name+"/input_line.txt"
fname3 = path+name+"/original_line.txt"
fname5 = path+name+"/dataframe_withsandhi.txt"
fname6 = path+name+"/conflicts.csv"
fname7 = path+name+"/problem_words.txt"

In [152]:
def printoriginalline(fname) :
    with open(fname,encoding = 'utf-8' ,mode ='r') as fr :
        for line in fr.readlines() :
            if line.startswith( 'SENT: '):
                print(line)
                break
            print(line)
    return

printoriginalline(fname3)



filename : kAraka_viBakwi_sents_GOLD_simple.txt

lineno : 2719

SENT: rAmaH arGyeNa qRiM pUjayawi



In [146]:
def printlines(fname) :
    with open(fname,encoding = 'utf-8' ,mode ='r') as fr :
        for line in fr.readlines() :
            print(line)
    return


printlines(fname2)


input line : vyAGraH kirIn awwi

converted line : vyāghraḥ kirīn atti 

converted line char :v,y,ā,gh,r,a,ḥ, ,k,i,r,ī,n, ,a,t,t,i, 



In [164]:
def undefinedwords(fname) :
    pbwords = []
    with open(fname,encoding = 'utf-8' ,mode ='r') as fr :
        lines = fr.readlines() 
        for line in lines[1:]:
            pbwords.append(line[:-1])
    return pbwords
undefinedwords(fname7)

['arghyeṇa_ṛṣim']

In [148]:
def dependency_structure(fname):
    ds = pd.read_csv(fname,encoding = 'utf-8' ,sep = '\t',skiprows = 4,header=None)
    return ds


ds = dependency_structure(fname3)
ds

,0,1,2,3
0,1,vyAGraH,NP,"karwA,3"
1,2,kirIn,NaN,"karma,3"
2,3,awwi,NaN,NaN


In [149]:
def fetched_data(fname) :
    fd = pd.read_csv(fname,encoding = 'utf-8' ,sep = ',')
    return fd


fd = fetched_data(fname5)
fd

,id,level,color_class,position,chunk_no,word,lemma,pre_verb,morph,colspan,wordlenth,aux_inf,word_slp1,sandhi_indexs,sandhi_words
0,0,1,deep_sky_back,0,1,vyāghraḥ,vyāghra,NaN,nom. sg. m.,7,8,NaN,vyAGraH,NaN,NaN
1,1,1,grey_back,8,2,kirīn,kirīn,NaN,?,5,5,NaN,kirIn,NaN,NaN
2,2,1,carmin_back,14,3,atti,ad,NaN,pr. [2] ac. sg. 3,4,4,sence of lemma = 1,atti,NaN,NaN


In [70]:
def conflicts_dataframe(fname) :
    cd = pd.read_csv(fname,encoding = 'utf-8' ,sep = ',')
    return cd


cd = conflicts_data(fname6)
cd
## here 2 == conflict ,1 = sandhi possible  , 0 = not conflict
## same word in same level , same position with different index are not taken as conflict (as it represent same word only)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0,0,0,0,0,0,0,0,2,0,...,2,2,0,2,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,1,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,1,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,2
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2,0,0,0,0,0,0,0,0,0,...,1,1,0,1,0,0,0,0,0,0
9,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
## here we give index as input and get conflicted words dataframe
def conflict_for_index(i):
    ci = fd.loc[cd.loc[cd[str(i)] == 2].index ]
    return ci

ci = conflict_for_index(8)
ci
## here we only printed conflicted ie checked value == 2

,id,level,color_class,position,chunk_no,word,lemma,pre_verb,morph,colspan,wordlenth,aux_inf,word_slp1,sandhi_indexs,sandhi_words
0,0,1,light_blue_back,0,1,anekāsu,aneka,NaN,loc. pl. f.,7,7,NaN,anekAsu,NaN,NaN
13,13,3,light_blue_back,0,1,anekā,aneka,NaN,nom. sg. f.,5,5,NaN,anekA,"19, 20, 22,","asu(asu), asu(asu), āsu(Asu),"
15,15,4,yellow_back,0,1,an,an,NaN,iic.,2,2,sence of lemma = 1,an,NaN,NaN
16,16,4,yellow_back,2,1,eka,eka,NaN,iic.,3,3,NaN,eka,"19, 20, 22,","asu(asu), asu(asu), āsu(Asu),"


In [250]:
def inputline_(fname) :
    with open(fname,encoding = 'utf-8' ,mode ='r') as fr :
        for line in fr.readlines() :
            if(line.startswith("input line : ")):
                return line[13:-1]


In [251]:
def originalline_(fname) :
    with open(fname,encoding = 'utf-8' ,mode ='r') as fr :
        for line in fr.readlines() :
            if(line.startswith("SENT: ")):
                return line[6:-1]

In [252]:
def undefinedwordsandlines(fpath,name) :
    pbwords = pd.DataFrame(columns = ['pbword','inputline','originalline','folder','path'])
#     path = "workdata/lines_submit/wrong_lines/sorteddata/"
    fname = fpath+name +"/problem_words.txt"
    with open(fname,encoding = 'utf-8' ,mode ='r') as fr :
        lines = fr.readlines() 
        for line in lines[1:]:
            pb = pd.Series([line[:-1],inputline_(fpath+name +"/input_line.txt"),originalline_(fpath+name +"/original_line.txt") , name ,fpath], index=['pbword','inputline','originalline','folder','path'])
            
            pbwords = pbwords.append(pb, ignore_index=True)
            
    return pbwords


In [253]:
pbwords = pd.DataFrame(columns = ['pbword','inputline','originalline','folder','path'])
with open("workdata/lines_submit/modified_wrong_lines/analasys/no_needtomodify_folders.txt",encoding = 'utf-8' ,mode ='r') as fr :
#     print(len(fr.readlines()))
    for line in fr.readlines() :
#         pbwords.extend(undefinedwords(fname = path+line[:-1]+"/problem_words.txt"))
        pb = undefinedwordsandlines('workdata/lines_submit/wrong_lines/sorteddata/',line[:-1])
#         display(pb)
        pbwords = pbwords.append(pb, ignore_index=True)
        
pbwords 


,pbword,inputline,originalline,folder,path
0,tasthivaḥ,nagarasya samIpe wasWivaH rAjasEnyaM apaSyaw,nagarasya samIpe wasWivaH rAja-sEnyaM apaSyaw,1039,workdata/lines_submit/wrong_lines/sorteddata/
1,jānami,wava ASayam na jAnami,wava ASayam na jAnami,1048,workdata/lines_submit/wrong_lines/sorteddata/
2,bhitvā,rAwrinakRawrarASim BiwvA prasaran aruNoxayakir...,rAwrinakRawrarASim BiwvA prasaran aruNoxayakir...,1058,workdata/lines_submit/wrong_lines/sorteddata/
3,bhitvā,ewam SuBoxarkam xqRtvA vikasiwahqxayaH aXiraWa...,ewam SuBoxarkam xqRtvA vikasiwahqxayaH aXiraWa...,1066,workdata/lines_submit/wrong_lines/sorteddata/
4,parākramiṇām,parAkramiNAm paFca rAjapuwrANAm BAgyavawi mAwa...,parAkramiNAm paFca rAjapuwrANAm BAgyavawi mAwa...,1078,workdata/lines_submit/wrong_lines/sorteddata/
5,jāgṛti,kim ewAxqSaH eva svapnaH mayi jAgqwi eva axya ...,kim ewAxqSaH eva svapnaH mayi jAgqwi eva axya ...,1089,workdata/lines_submit/wrong_lines/sorteddata/
6,uttaraya,na me praSnAn uwwaraya,na me praSnAn uwwaraya,1109,workdata/lines_submit/wrong_lines/sorteddata/
7,vināśit,kevalam ixam wAvaw kaWaya yaw wvayA eva vinASi...,kevalam ixam wAvaw kaWaya yaw wvayA eva vinASi...,1111,workdata/lines_submit/wrong_lines/sorteddata/
8,druṣṭam,kevalam ixam wAvaw kaWaya yaw wvayA eva vinASi...,kevalam ixam wAvaw kaWaya yaw wvayA eva vinASi...,1111,workdata/lines_submit/wrong_lines/sorteddata/
9,itopi,iwopi aXikam vaMSagOravam na apekRe,iwopi aXikam vaMSagOravam na apekRe,1122,workdata/lines_submit/wrong_lines/sorteddata/


In [254]:
foldnames = os.listdir("workdata/lines_submit/modified_wrong_lines/wrong_lines/sorteddata/")
for foldname in foldnames:
        if foldname == ".ipynb_checkpoints":
            continue
        pb = undefinedwordsandlines('workdata/lines_submit/modified_wrong_lines/wrong_lines/sorteddata/',foldname)
#         display(pb)
        pbwords = pbwords.append(pb, ignore_index=True)
pbwords

,pbword,inputline,originalline,folder,path
0,tasthivaḥ,nagarasya samIpe wasWivaH rAjasEnyaM apaSyaw,nagarasya samIpe wasWivaH rAja-sEnyaM apaSyaw,1039,workdata/lines_submit/wrong_lines/sorteddata/
1,jānami,wava ASayam na jAnami,wava ASayam na jAnami,1048,workdata/lines_submit/wrong_lines/sorteddata/
2,bhitvā,rAwrinakRawrarASim BiwvA prasaran aruNoxayakir...,rAwrinakRawrarASim BiwvA prasaran aruNoxayakir...,1058,workdata/lines_submit/wrong_lines/sorteddata/
3,bhitvā,ewam SuBoxarkam xqRtvA vikasiwahqxayaH aXiraWa...,ewam SuBoxarkam xqRtvA vikasiwahqxayaH aXiraWa...,1066,workdata/lines_submit/wrong_lines/sorteddata/
4,parākramiṇām,parAkramiNAm paFca rAjapuwrANAm BAgyavawi mAwa...,parAkramiNAm paFca rAjapuwrANAm BAgyavawi mAwa...,1078,workdata/lines_submit/wrong_lines/sorteddata/
5,jāgṛti,kim ewAxqSaH eva svapnaH mayi jAgqwi eva axya ...,kim ewAxqSaH eva svapnaH mayi jAgqwi eva axya ...,1089,workdata/lines_submit/wrong_lines/sorteddata/
6,uttaraya,na me praSnAn uwwaraya,na me praSnAn uwwaraya,1109,workdata/lines_submit/wrong_lines/sorteddata/
7,vināśit,kevalam ixam wAvaw kaWaya yaw wvayA eva vinASi...,kevalam ixam wAvaw kaWaya yaw wvayA eva vinASi...,1111,workdata/lines_submit/wrong_lines/sorteddata/
8,druṣṭam,kevalam ixam wAvaw kaWaya yaw wvayA eva vinASi...,kevalam ixam wAvaw kaWaya yaw wvayA eva vinASi...,1111,workdata/lines_submit/wrong_lines/sorteddata/
9,itopi,iwopi aXikam vaMSagOravam na apekRe,iwopi aXikam vaMSagOravam na apekRe,1122,workdata/lines_submit/wrong_lines/sorteddata/


In [255]:
pbw = pd.DataFrame(columns = ['pbword','inputline','originalline','folder','path'])
with open("workdata/lines_submit/modified_wrong_lines/analasys/errormodified_folders.txt",encoding = 'utf-8' ,mode ='r') as fr :
    for line in fr.readlines() :
        pb = undefinedwordsandlines('workdata/lines_submit/wrong_lines/sorteddata/',line[:-1])
#         display(pb)
        pbw = pbw.append(pb, ignore_index=True)
pbw

,pbword,inputline,originalline,folder,path
0,gatvā_vidyā adhyayanam,aham vixyAlayam gawvA vixyAaXyayanam kariRyAmi,aham vixyAlayam gawvA vixyA-aXyayanam kariRyAmi,268,workdata/lines_submit/wrong_lines/sorteddata/
1,paṅkilā_bhavati,varRAkAle vqRtyA BUmiH paMkilA Bavawi,varRAkAle vqRtyA BUmiH paMkilA Bavawi,354,workdata/lines_submit/wrong_lines/sorteddata/
2,niṣaṇṇā_devīraktacandanadhāriṇā,pravAwaSayane niRaNNA xevI rakwacanxanaXAriNA ...,pravAwaSayane niRaNNA xevI rakwacanxanaXAriNA ...,426,workdata/lines_submit/wrong_lines/sorteddata/
3,svarṇasya_ābharaṇam,waxA saH vAyasaH waw svarNasya ABaraNam caMcvA...,waxA saH vAyasaH waw svarNasya ABaraNam caMcvA...,44,workdata/lines_submit/wrong_lines/sorteddata/
4,cañcvā_dhārayitvā_ānayati,waxA saH vAyasaH waw svarNasya ABaraNam caMcvA...,waxA saH vAyasaH waw svarNasya ABaraNam caMcvA...,44,workdata/lines_submit/wrong_lines/sorteddata/
5,gāṇḍivinā_evam,awrAnware SakwiKaMdAmarRiwena gAMdivinA evaM B...,awrAnware SakwiKaMdAmarRiwena gAMdivinA evaM B...,493,workdata/lines_submit/wrong_lines/sorteddata/


In [256]:
pbwords.to_csv("workdata/lines_submit/modified_wrong_lines/analasys/problemwords.csv" ,encoding = 'utf-8' , sep=',',index = False,mode = 'w' )
pbw.to_csv("workdata/lines_submit/modified_wrong_lines/analasys/doubtwords.csv" ,encoding = 'utf-8' , sep=',',index = False,mode = 'w' )

In [257]:
pbw['pbword_wx/transcoder'] = pbw['pbword']
pbw['inputlne_roman/transcoder'] = pb['inputline']
for i in pbw.index:
    x = transcoder.transcoder_processString(pbw.loc[i,'pbword'],'roman','slp1')
    y = transcoder.transcoder_processString(pbw.loc[i,'inputline'],'wx','slp1')
    pbw.loc[i,'pbword_wx/transcoder']   =  transcoder.transcoder_processString(x,'slp1','wx')
    pbw.loc[i,'inputlne_roman/transcoder']   =  transcoder.transcoder_processString(y,'slp1','roman')


In [258]:
pbw.to_csv("workdata/lines_submit/modified_wrong_lines/analasys/doubtwords_withtrancodedword.csv" ,encoding = 'utf-8' , sep=',',index = False,mode = 'w' )

In [267]:
pbwords.pbword.value_counts()
print(pbwords.pbword.nunique())
len(pbwords)

632


773

In [261]:
with open("workdata/lines_submit/modified_wrong_lines/analasys/modified_folders.txt",encoding = 'utf-8' ,mode ='r') as fr :
     print(len(fr.readlines()))
265+115

380


380

In [264]:
with open("workdata/lines_submit/modified_wrong_lines/analasys/no_needtomodify_folders.txt",encoding = 'utf-8' ,mode ='r') as fr :
     print(len(fr.readlines()))
380+555+5


555


940

In [86]:
pbwords = []
with open("workdata/lines_submit/analasys/undifined_words.txt",encoding = 'utf-8' ,mode ='r') as fr :
    for w in fr.readlines():
        pbwords.append(w[:-1])

In [95]:
i =0
ws = []
for w in pbwords :
    if not (re.findall(r'_',w)) :
        if w not in ws :
            ws.append(w)
            i = i + 1
        

In [96]:
i

587

In [124]:
x=['1','2','3']
x.append(x)

In [125]:
print(x)

['1', '2', '3', [...]]
